#### In this notebook, I will workout of different ways of table creation and loading strategies 

#### creating table with orc format

In [82]:
import org.apache.spark.sql.SparkSession


lastException: Throwable = null


In [83]:
val spark = SparkSession.
    builder.
    config("spark.ui.port", "0").
    config("spark.sql.warehouse.dir", "/user/itv001389/warehouse").
    enableHiveSupport.
    master("yarn").
    appName("Getting Started - Spark with Scala").
    getOrCreate

spark = org.apache.spark.sql.SparkSession@7acb4298


org.apache.spark.sql.SparkSession@7acb4298

In [20]:
spark.sql("create database Raj")

[]

In [22]:
spark.sql("use Raj")

lastException: Throwable = null


[]

In [23]:
val df = spark.read.format("csv").schema("order_id int ,Order_date timestamp ,Order_product_id int ,Order_status string")
.load("hdfs://m01.itversity.com:9000/user/itv001389/retail_db/orders")

df = [order_id: int, Order_date: timestamp ... 2 more fields]


[order_id: int, Order_date: timestamp ... 2 more fields]

In [24]:
df.write.saveAsTable("Raj.Orders")

In [25]:
val df1 = spark.read.format("csv").schema("order_item_id int,order_item_order_id int,order_item_product_id int,order_item_quantity int,order_item_subtotal decimal(17,4),order_item_product_price decimal(17,4)")
.load("hdfs://m01.itversity.com:9000/user/itv001389/retail_db/order_items")

df1 = [order_item_id: int, order_item_order_id: int ... 4 more fields]


[order_item_id: int, order_item_order_id: int ... 4 more fields]

In [26]:
df1.write.saveAsTable("Raj.Order_items")

In [28]:
spark.sql("show tables").show()

+--------+-----------+-----------+
|database|  tableName|isTemporary|
+--------+-----------+-----------+
|     raj|order_items|      false|
|     raj|     orders|      false|
+--------+-----------+-----------+



In [84]:
spark.sql("select * from Raj.Orders limit 3").show()

+--------+-------------------+----------------+---------------+
|order_id|         Order_date|Order_product_id|   Order_status|
+--------+-------------------+----------------+---------------+
|       1|2013-07-25 00:00:00|           11599|         CLOSED|
|       2|2013-07-25 00:00:00|             256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:00|           12111|       COMPLETE|
+--------+-------------------+----------------+---------------+



In [33]:
spark.sql("describe formatted Raj.Orders").show(false)

+----------------------------+--------------------------------------------------------------------+-------+
|col_name                    |data_type                                                           |comment|
+----------------------------+--------------------------------------------------------------------+-------+
|order_id                    |int                                                                 |null   |
|Order_date                  |timestamp                                                           |null   |
|Order_product_id            |int                                                                 |null   |
|Order_status                |string                                                              |null   |
|                            |                                                                    |       |
|# Detailed Table Information|                                                                    |       |
|Database                   

In [34]:
spark.sql("select * from Raj.Order_items limit 3").show()

+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|            1|                  1|                  957|                  1|           299.9800|                299.9800|
|            2|                  2|                 1073|                  1|           199.9900|                199.9900|
|            3|                  2|                  502|                  5|           250.0000|                 50.0000|
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+



In [35]:
spark.sql("describe formatted Raj.Order_items").show(false)

+----------------------------+-------------------------------------------------------------------------+-------+
|col_name                    |data_type                                                                |comment|
+----------------------------+-------------------------------------------------------------------------+-------+
|order_item_id               |int                                                                      |null   |
|order_item_order_id         |int                                                                      |null   |
|order_item_product_id       |int                                                                      |null   |
|order_item_quantity         |int                                                                      |null   |
|order_item_subtotal         |decimal(17,4)                                                            |null   |
|order_item_product_price    |decimal(17,4)                                                     

#### Creating Hive table using ORC format

##### ORC and Parquet are the best file formats in Hadoop environment. They are columnar storage. By default this file formats have inbuilt intelligence because of predicate and projection pushdown

##### For Eg : When we want to retrieve specific value then in this file formats it will go and directly get that particular value without scanning entire file 

In [37]:
spark.sql("CREATE TABLE Raj.order_orc( Order_id int, Order_date timestamp, Order_product_id int, order_status STRING) stored as orc location 'hdfs://m01.itversity.com:9000/user/itv001389/warehouse/raj.db/order_orc'")

[]

In [38]:
spark.sql("select * from Raj.order_orc limit 5").show(false)

+--------+----------+----------------+------------+
|Order_id|Order_date|Order_product_id|order_status|
+--------+----------+----------------+------------+
+--------+----------+----------------+------------+



#### Loading data into orc table

In [45]:
spark.sql("insert overwrite  table Raj.order_orc select * from  Raj.Orders")

lastException: Throwable = null


[]

In [108]:
spark.sql("describe formatted Raj.Orders").show(false)

+----------------------------+--------------------------------------------------------------------+-------+
|col_name                    |data_type                                                           |comment|
+----------------------------+--------------------------------------------------------------------+-------+
|order_id                    |int                                                                 |null   |
|Order_date                  |timestamp                                                           |null   |
|Order_product_id            |int                                                                 |null   |
|Order_status                |string                                                              |null   |
|                            |                                                                    |       |
|# Detailed Table Information|                                                                    |       |
|Database                   

In [46]:
spark.sql("select count(*) from Raj.order_orc").show()

+--------+
|count(1)|
+--------+
|   68883|
+--------+



In [47]:
spark.sql("select * from Raj.order_orc limit 2").show()

+--------+-------------------+----------------+---------------+
|Order_id|         Order_date|Order_product_id|   order_status|
+--------+-------------------+----------------+---------------+
|       1|2013-07-25 00:00:00|           11599|         CLOSED|
|       2|2013-07-25 00:00:00|             256|PENDING_PAYMENT|
+--------+-------------------+----------------+---------------+



In [104]:
spark.sql("describe formatted Raj.order_orc").show(false)

+----------------------------+-----------------------------------------------------------------------+-------+
|col_name                    |data_type                                                              |comment|
+----------------------------+-----------------------------------------------------------------------+-------+
|Order_id                    |int                                                                    |null   |
|Order_date                  |timestamp                                                              |null   |
|Order_product_id            |int                                                                    |null   |
|order_status                |string                                                                 |null   |
|                            |                                                                       |       |
|# Detailed Table Information|                                                                       |       |
|

#### CTAS(Create Table As Select)

In [ ]:
spark.sql("create table Raj.order_parq1 as select * from Raj.Orders")

## Table properties

##### Converting managed table to external table using property

In [ ]:
spark.sql("alter table Raj.order_orc SET TBLPROPERTIES('EXTERNAL'='TRUE')")

##### Creating a table by using skipping header property

In [95]:
spark.sql("create table Raj.order_skip_header3(order_id int,order_date timestamp,order_product_id int,order_status string) stored as parquet location 'hdfs://m01.itversity.com:9000/user/itv001389/warehouse/raj.db/orders' TBLPROPERTIES('skip.header.line.count'='100') ")

[]

In [90]:
spark.sql("describe formatted Raj.Orders").show(false)

+----------------------------+--------------------------------------------------------------------+-------+
|col_name                    |data_type                                                           |comment|
+----------------------------+--------------------------------------------------------------------+-------+
|order_id                    |int                                                                 |null   |
|Order_date                  |timestamp                                                           |null   |
|Order_product_id            |int                                                                 |null   |
|Order_status                |string                                                              |null   |
|                            |                                                                    |       |
|# Detailed Table Information|                                                                    |       |
|Database                   

In [96]:
spark.sql("select * from  Raj.order_skip_header3 limit 2").show()

+--------+-------------------+----------------+---------------+
|order_id|         order_date|order_product_id|   order_status|
+--------+-------------------+----------------+---------------+
|       1|2013-07-25 00:00:00|           11599|         CLOSED|
|       2|2013-07-25 00:00:00|             256|PENDING_PAYMENT|
+--------+-------------------+----------------+---------------+



In [97]:
spark.sql("select count(*) from Raj.order_skip_header3 ").show()

+--------+
|count(1)|
+--------+
|   68883|
+--------+



#####   Creating a table by using immutable property
##### Immutable property is used to avoid the duplicate data getting loaded into the table

In [70]:
spark.sql("create table Raj.order_immu(order_id int,order_date timestamp,order_product_id int,order_status string) row format delimited fields terminated by ',' location 'hdfs://m01.itversity.com:9000/user/itv001389/warehouse/raj.db/order_imm' TBLPROPERTIES('immutable'='true') ")

[]

In [102]:
spark.sql("insert into table Raj.order_immu select * from Raj.Orders")

[]

In [ ]:
spark.sql("select count(*) from Raj.order_immu ").show()

In [ ]:
spark.sql("select * from Orders limit 2").show()

#####    Creating a table using snappy compression 

In [107]:
spark.sql("create table Raj.order_snap(order_id int,order_date timestamp,order_product_id int,order_status string) stored as orc TBLPROPERTIES('compression'='snappy')")

lastException: Throwable = null


[]

In [ ]:
spark.sql("insert into table Raj.order_snap select * from Raj.Orders").show(false)

In [110]:
spark.sql("select count(*) from Raj.order_snap ").show()

+--------+
|count(1)|
+--------+
|   68883|
+--------+



In [111]:
spark.sql("select * from Raj.order_snap limit 5").show(false)

+--------+-------------------+----------------+---------------+
|order_id|order_date         |order_product_id|order_status   |
+--------+-------------------+----------------+---------------+
|1       |2013-07-25 00:00:00|11599           |CLOSED         |
|2       |2013-07-25 00:00:00|256             |PENDING_PAYMENT|
|3       |2013-07-25 00:00:00|12111           |COMPLETE       |
|4       |2013-07-25 00:00:00|8827            |CLOSED         |
|5       |2013-07-25 00:00:00|11318           |COMPLETE       |
+--------+-------------------+----------------+---------------+

